# Analyse de données de systèmes éducatifs

Nous travaillons sur 5 fichiers CSV qui contiennent des données sur les systèmes éducatifs de différents pays. 

Chaque fichier est considéré comme une table et l'ensemble des fichiers comme une base de données.

Nous allons donc commencer par essayer de comprendre les données qu'ils contiennent puis de charger les données et les explorer.

## Objectifs

1. Charger les données
2. Explorer les données
3. Nettoyer les données
4. Analyser les données
5. Visualiser les données
6. Interpréter les résultats
7. Conclusion et réponses aux problématiques

## Import des librairies nécessaires

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [12]:
# Fonction pour afficher les informations sur les données (récupérée d'un projet existant)
def infos (df):
    nb_li = df.shape[0]
    nb_co = df.shape[1]
    t = np.empty(nb_li)
    t.fill(nb_li)
    df_l_null = pd.DataFrame(df.T.isna().sum()) # tableau du nbe de nul par lignes (+800 000 lignes)
    df_c_null = pd.DataFrame(df.isna().sum()) # tableau du nbe de nul par colonnes (+65 colonnes)
    
    # nbe de lignes sans 'null'
    al = len([x for x in df_l_null[0] if x==0])
    nb_ss_null = pd.DataFrame([al]).rename(index={0:'lign_ss_null'}).T
    pct_ss_null = pd.DataFrame([al*100/nb_li]).rename(index={0:'lign_ss_null'}).T
    # nbe de lignes 'null'
    bl = len([x for x in df_l_null[0] if x==nb_co])
    nb_null = pd.DataFrame([bl]).rename(index={0:'lign_null'}).T
    pct_null = pd.DataFrame([bl*100/nb_li]).rename(index={0:'lign_null'}).T
    # nbe de lignes mixtes
    cl = len([x for x in df_l_null[0] if (x!=0 and x!=nb_co)])
    nb_mix = pd.DataFrame([cl]).rename(index={0:'lign_mix'}).T
    pct_mix = pd.DataFrame([cl*100/nb_li]).rename(index={0:'lign_mix'}).T
    infos_nb = pd.concat([nb_ss_null, nb_null, nb_mix],axis=1, sort=False).rename(index={0:'nb'})
    infos_pct = pd.concat([pct_ss_null, pct_null, pct_mix],axis=1, sort=False).rename(index={0:'pct'})
    infos_l = pd.concat([infos_nb,infos_pct], sort=False)
    # nbe de lignes total
    infos_l["lign_tot"] = [infos_l.T['nb'].sum(), infos_l.T['pct'].sum()]
    
    # nbe de colonnes sans 'null'
    ac = len([x for x in df_c_null[0] if x==0])
    nb_ss_null = pd.DataFrame([ac]).rename(index={0:'col_ss_null'}).T
    pct_ss_null = pd.DataFrame([ac*100/nb_co]).rename(index={0:'col_ss_null'}).T
    # nbe de colonnes 'null'
    bc = len([x for x in df_c_null[0] if x==nb_li])
    nb_null = pd.DataFrame([bc]).rename(index={0:'col_null'}).T
    pct_null = pd.DataFrame([bc*100/nb_co]).rename(index={0:'col_null'}).T
    # nbe de colonnes mixtes
    cc = len([x for x in df_c_null[0] if (x!=0 and x!=nb_li)])
    nb_mix = pd.DataFrame([cc]).rename(index={0:'col_mix'}).T
    pct_mix = pd.DataFrame([cc*100/nb_co]).rename(index={0:'col_mix'}).T
    infos_nb = pd.concat([nb_ss_null, nb_null, nb_mix],axis=1, sort=False).rename(index={0:'nb'})
    infos_pct = pd.concat([pct_ss_null, pct_null, pct_mix],axis=1, sort=False).rename(index={0:'pct'})
    infos_c = pd.concat([infos_nb,infos_pct], sort=False)
    # nbe de lignes total
    infos_c["col_tot"] = [infos_c.T['nb'].sum(), infos_c.T['pct'].sum()]
    
    infos = pd.concat([infos_l,infos_c], axis=1, sort=False)
    
    return infos

## Chargement des données

In [15]:
# Le dropna permet de supprimer les colonnes fantômes "Unamed" en fin de tableau
country_series = pd.read_csv('CSV/EdStatsCountry-Series.csv').dropna(how='all', axis='columns')
country = pd.read_csv('CSV/EdStatsCountry.csv').dropna(how='all', axis='columns')
data = pd.read_csv('CSV/EdStatsData.csv').dropna(how='all', axis='columns')
footnote = pd.read_csv('CSV/EdStatsFootNote.csv').dropna(how='all', axis='columns')
series = pd.read_csv('CSV/EdStatsSeries.csv').dropna(how='all', axis='columns')

# Création d'autres dataframes pour la partie nettoyage des données
country_series_c = country_series.copy()
country_c = country.copy()
data_c = data.copy()
footnote_c = footnote.copy()
series_c = series.copy()

## Exploration des colonnes des tables

### Table **EdStatsCountry-Series** :

Cette table représente les séries d'indicateurs pour chaque pays.

- **CountryCode** -> ISO 3166-1 alpha-3 : les codes d'identification ISO des différents pays du monde 
- Indication du **type de données** étudiées :
    - "**SP.POP.TOTL**" : Total population.
    - "**SP.POP.GROW**" : Population growth (annual %).
    - "**NY.GDP.PCAP.PP.CD**" : GDP (PIB) per capita, PPP (Parité de pouvoir d'achat) (current international $).
    - "**NY.GDP.MKTP.PP.KD**" : GDP, PPP (constant 2011 international $).
    - "**NY.GNP.MKTP.PP.CD**" : GNI (Revenu national brut), PPP (current international $).
    - "**NY.GDP.MKTP.PP.CD**" : GDP, PPP (current international $).
    - "**NY.GDP.PCAP.PP.KD**" : GDP per capita, PPP (constant 2011 international $).
    - "**NY.GNP.PCAP.PP.CD**" : GNI per capita, PPP (current international $).
    - "**SP.POP.1564.TO.ZS**" : Population ages 15-64 (% of total).
    - "**SP.POP.0014.TO.ZS**" : Population ages 0-14 (% of total).
    - "**.MA.ZS**" : Population, male (% of total).
    - "**.FE.ZS**" : Population, female (% of total).
    - "**NY.GNP.PCAP.CD**" : GNI per capita, Atlas method (current US$).
    - "**NY.GDP.PCAP.CD**" : GDP per capita (current US$).
    - "**NY.GDP.PCAP.KD**" : GDP per capita (constant 2010 US$).
    - "**SP.POP.1564.MA.IN**" : Population ages 15-64, male.
    - "**SP.POP.0014.TO**": Population ages 0-14, total.
    - "**SP.POP.1564.TO**" : Population ages 15-64, total.
    - "**SP.POP.1564.FE.IN**" : Population ages 15-64, female.
    - "**SP.POP.0014.MA.IN**" : Population ages 0-14, male.
    - "**SP.POP.0014.FE.IN**" : Population ages 0-14, female.
- **Description** et informations sur la source de données

### Table **EdStatsCountry** :

Cette table contient des informations sur les pays.

- **Country Code** -> ISO 3166-1 alpha-3 : les codes d'identification ISO des différents pays du monde
- **Short Name** : Nom court du pays
- **Table Name** : Nom du pays dans la table
- **Long Name** : Nom complet du pays
- **2-alpha code** : Code alpha-2 du pays
- **Currency Unit** : Unité monétaire du pays
- **Special Notes** : Notes spéciales sur le pays
- **Region** : Région du monde
- **Income Group** : Groupe de revenu
- **WB-2 code** : Code WB-2 du pays
- **National accounts base year** : Année de base des comptes nationaux
- **National accounts reference year** : Année de référence des comptes nationaux
- **SNA price valuation** : Evaluation des prix SNA (méthode d'évaluation des prix utilisée dans le cadre du système de comptabilité nationale)
- **Lending category** : Catégorie de prêt
- **Other groups** : Autres groupes
- **System of National Accounts** : Système de comptabilité nationale
- **Alternative conversion factor** : Facteur de conversion alternatif
- **PPP survey year** : Année de l'enquête sur la parité de pouvoir d'achat
- **Balance of Payments Manual in use** : Manuel de la balance des paiements en cours d'utilisation
- **External debt Reporting status** : Statut de déclaration de la dette extérieure
- **System of trad**e : Système de commerce
- **Government Accounting concep**t : Concept de comptabilité gouvernementale
- **IMF data dissemination standard** : Norme de diffusion des données du FMI
- **Latest population census** : Dernier recensement de la population
- **Latest household survey** : Dernière enquête auprès des ménages
- **Source of most recent Income and expenditure data** : Source des données les plus récentes sur les revenus et les dépenses
- **Vital registration complete** : Enregistrement vital complet
- **Latest agricultural census** : Dernier recensement agricole
- **Latest industrial data** : Dernières données industrielles
- **Latest trade data** : Dernières données commerciales
- **Latest water withdrawal data** : Dernières données sur les prélèvements d'eau

### Table **EdStatsData** :

Cette table contient les données des indicateurs pour chaque pays.

- **Country Name** : Nom du pays
- **Country Code** : Code du pays
- **Indicator Name** : Nom de l'indicateur
- **Indicator Code** : Code de l'indicateur
- **1960 - 2015** : Valeurs de l'indicateur pour chaque année
- **2016 - 2100** : Prévisions de l'indicateur pour chaque année

### Table **EdStatsFootNote** :

Cette table contient des notes de bas de page pour les indicateurs.

- **CountryCode** : Code du pays
- **SeriesCode** : Code de la série
- **Year** : Année
- **Description** : Description de la note de bas de page

### Table **EdStatsSeries** :

Cette table contient des informations sur les séries d'indicateurs.

- **Series Code** : Code de la série
- **Topic** : Thème
- **Indicator Name** : Nom de l'indicateur
- **Short definition** : Définition courte
- **Long definition** : Définition longue
- **Unit of measure** : Unité de mesure
- **Periodicity** : Périodicité
- **Base Period** : Période de base
- **Other notes** : Autres notes
- **Aggregation method** : Méthode d'agrégation
- **Limitations and exceptions** : Limitations et exceptions
- **Notes from original source** : Notes de la source originale
- **General comments** : Commentaires généraux
- **Source** : Source
- **Statistical concept and methodology** : Concept statistique et méthodologie
- **Development relevance** : Pertinence du développement
- **Related source links** : Liens de source connexes
- **Other web links** : Autres liens web
- **Related indicators** : Indicateurs connexes
- **License Type** : Type de licence

## Exploration des données

### Analyse des valeurs manquantes ou nulles dans les tables

In [11]:
## Nbre de null par ligne et par colonne
infos_t = pd.concat([infos(data), infos(country), infos(series),\
                  infos(country_series), infos(footnote)], axis = 0,\
                 keys=['data', 'country', 'series', 'cnt_ser', 'footnote'])
pd.options.display.float_format = '{:.1f}'.format
infos_t

lign_ss_null  lign_null  lign_mix  lign_tot  col_ss_null  \
data     nb            0.0        0.0  886930.0  886930.0          4.0   
         pct           0.0        0.0     100.0     100.0          5.7   
country  nb            0.0        0.0     241.0     241.0          4.0   
         pct           0.0        0.0     100.0     100.0         12.5   
series   nb            0.0        0.0    3665.0    3665.0          5.0   
         pct           0.0        0.0     100.0     100.0         23.8   
cnt_ser  nb            0.0        0.0     613.0     613.0          3.0   
         pct           0.0        0.0     100.0     100.0         75.0   
footnote nb            0.0        0.0  643638.0  643638.0          4.0   
         pct           0.0        0.0     100.0     100.0         80.0   

              col_null  col_mix  col_tot  
data     nb        1.0     65.0     70.0  
         pct       1.4     92.9    100.0  
country  nb        1.0     27.0     32.0  
         pct       3.1     84.4    100.0  
series   nb        6.0     10.0     21.0  
         pct      28.6     47.6    100.0  
cnt_ser  nb        1.0      0.0      4.0  
         pct      25.0      0.0    100.0  
footnote nb        1.0      0.0      5.0  
         pct      20.0      0.0    100.0